In [2]:
!pip install warcio
!pip install transformers
!pip install keybert
!pip install spacy
!pip install textstat
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [3]:
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-26/warc.paths.gz


--2025-07-23 18:21:42--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-26/warc.paths.gz
Resolving data.commoncrawl.org (data.commoncrawl.org)... 216.137.52.3, 216.137.52.110, 216.137.52.112, ...
Connecting to data.commoncrawl.org (data.commoncrawl.org)|216.137.52.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201085 (196K) [binary/octet-stream]
Saving to: ‘warc.paths.gz’

warc.paths.gz       100%[===================>] 196.37K   334KB/s    in 0.6s    

2025-07-23 18:21:43 (334 KB/s) - ‘warc.paths.gz’ saved [201085/201085]



In [4]:
import gzip
with gzip.open('warc.paths.gz', 'rt') as f:
    warc_paths = [line.strip() for line in f.readlines()[:3]]

for path in warc_paths:
    print(path)


crawl-data/CC-MAIN-2025-26/segments/1749709481111.44/warc/CC-MAIN-20250612112840-20250612142840-00000.warc.gz
crawl-data/CC-MAIN-2025-26/segments/1749709481111.44/warc/CC-MAIN-20250612112840-20250612142840-00001.warc.gz
crawl-data/CC-MAIN-2025-26/segments/1749709481111.44/warc/CC-MAIN-20250612112840-20250612142840-00002.warc.gz


In [5]:
first_warc_url = "https://data.commoncrawl.org/" + warc_paths[0]
print(first_warc_url)


https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-26/segments/1749709481111.44/warc/CC-MAIN-20250612112840-20250612142840-00000.warc.gz


In [6]:
!wget {first_warc_url}


--2025-07-23 18:21:44--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-26/segments/1749709481111.44/warc/CC-MAIN-20250612112840-20250612142840-00000.warc.gz
Resolving data.commoncrawl.org (data.commoncrawl.org)... 216.137.52.3, 216.137.52.110, 216.137.52.112, ...
Connecting to data.commoncrawl.org (data.commoncrawl.org)|216.137.52.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857098534 (817M) [application/octet-stream]
Saving to: ‘CC-MAIN-20250612112840-20250612142840-00000.warc.gz’

CC-MAIN-20250612112 100%[===================>] 817.39M  14.9MB/s    in 58s     

2025-07-23 18:22:42 (14.1 MB/s) - ‘CC-MAIN-20250612112840-20250612142840-00000.warc.gz’ saved [857098534/857098534]



In [7]:
import os
import re
import sqlite3
from datetime import datetime
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
from transformers import pipeline
from keybert import KeyBERT
import spacy
import textstat
import nltk


try:
    nltk.data.find('corpora/cmudict')
except LookupError:
    nltk.download('cmudict')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


try:
    spacy.load("en_core_web_sm")
except OSError:
    os.system("python -m spacy download en_core_web_sm")


class Args:
    warc_dir = '/content'
    db_name = 'test_output.db'
args = Args()
sentiment_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)
fact_opinion_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
keyword_extractor = KeyBERT()
ner_model = spacy.load("en_core_web_sm")
sentiment_labels = ["positive news", "negative news"]
topic_labels = [
    'Politics', 'Business', 'Technology', 'Science', 'Health',
    'Sports', 'Entertainment', 'Lifestyle', 'Education', 'Environment',
    'Crime', 'Weather', 'Economy', 'Real Estate', 'Automotive', 'Travel'
]


def is_english(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        html_tag = soup.find('html')
        if html_tag and html_tag.get('lang') == 'en':
            return True
        meta_tag = soup.find('meta', {'http-equiv': 'Content-Language'})
        if meta_tag and meta_tag.get('content') == 'en':
            return True

        text = soup.get_text()
        common = {"the", "and", "is", "in", "it", "you", "that", "was", "for"}
        words = re.findall(r'\b\w+\b', text.lower())
        return sum(word in common for word in words) / len(words) > 0.5 if words else False
    except:
        return False

def save_to_db(cursor, record, conn):
    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_articles (
                url TEXT, url_timestamp TEXT, category TEXT, sentiment INTEGER,
                score REAL, named_entities TEXT, keywords TEXT,
                readability_score REAL, opinion_type TEXT
            )
        ''')
        cursor.execute('''
            INSERT INTO news_articles
            (url, url_timestamp, category, sentiment, score, named_entities, keywords, readability_score, opinion_type)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            record['url'], record['timestamp'], record['category'], record['sentiment'],
            record['score'], record['named_entities'], record['keywords'],
            record['readability'], record['opinion']
        ))
        conn.commit()
    except sqlite3.Error as e:
        print(f"[DB Error] {e}")


def analyze_warc_file(file_path, cursor, conn):
    try:
        with open(file_path, 'rb') as stream:
            for entry in ArchiveIterator(stream):
                warc_date = entry.rec_headers.get_header("WARC-Date")
                if not warc_date:
                    continue

                try:
                    html = entry.content_stream().read().decode(errors='ignore')
                    if not is_english(html):
                        continue

                    soup = BeautifulSoup(html, 'html.parser')
                    title = soup.title.string if soup.title else ''
                    headers = ' '.join([h.get_text() for h in soup.find_all(['h1', 'h2', 'h3'])])
                    full_text = f"{title} {headers}".strip()
                    url = entry.rec_headers.get_header("WARC-Target-URI")

                    # Truncate text for spaCy
                    max_length = ner_model.max_length
                    truncated_text = full_text[:max_length]

                    text_15 = ' '.join(truncated_text.split()[:15])
                    text_40 = ' '.join(truncated_text.split()[:40])

                    # Sentiment & Topic
                    sent_result = sentiment_classifier(text_15, sentiment_labels)
                    topic_result = sentiment_classifier(text_40, topic_labels)
                    sentiment = 1 if sent_result['labels'][0] == 'positive news' else 0
                    sentiment_score = sent_result['scores'][0]
                    topic = topic_result['labels'][0]

                    # NER
                    doc = ner_model(truncated_text)
                    named_entities = list(set(ent.text for ent in doc.ents))

                    # Keywords
                    keywords = keyword_extractor.extract_keywords(truncated_text, top_n=5)
                    keyword_texts = [kw[0] for kw in keywords]

                    # Readability
                    readability = textstat.flesch_reading_ease(truncated_text)

                    # Opinion vs Fact
                    opinion_result = fact_opinion_classifier(text_40)
                    opinion = "opinion" if opinion_result[0]['label'] == "NEGATIVE" else "fact"

                    # Time
                    dt = datetime.fromisoformat(warc_date.replace('Z', '+00:00'))
                    formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')

                    # Save
                    record = {
                        'url': url,
                        'timestamp': formatted_time,
                        'category': topic,
                        'sentiment': sentiment,
                        'score': sentiment_score,
                        'named_entities': ', '.join(named_entities),
                        'keywords': ', '.join(keyword_texts),
                        'readability': readability,
                        'opinion': opinion
                    }
                    save_to_db(cursor, record, conn)

                except Exception as entry_err:
                    print(f"[Entry Error] {entry_err}")
    except Exception as e:
        print(f"[WARC Error] {e}")


def scan_directory(path, conn):
    cur = conn.cursor()
    for root, _, files in os.walk(path):
        for fname in files:
            if fname.endswith(".warc.gz"):
                fpath = os.path.join(root, fname)
                print(f"Processing: {fpath}")
                analyze_warc_file(fpath, cur, conn)

try:
    db_conn = sqlite3.connect(args.db_name)
    scan_directory(args.warc_dir, db_conn)
    db_conn.close()
except Exception as final_err:
    print(f"[Fatal Error] {final_err}")

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html, 'html.parser')
/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might w

[Entry Error] You must include at least one label and at least one sequence.


/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html, 'html.parser')
/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might w

[Entry Error] You must include at least one label and at least one sequence.
[Entry Error] You must include at least one label and at least one sequence.


/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html, 'html.parser')
/tmp/ipython-input-7-96948205.py:48: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might w

In [8]:
import sqlite3
conn = sqlite3.connect('test_output.db')
cursor = conn.cursor()
cursor.execute("select name from sqlite_master WHERE type='table';")
print(cursor.fetchall())


for row in cursor.execute("select * from  news_articles"):
    print(row)

conn.close()


[('news_articles',)]
('http://0577007.com/gsjj/', '2025-06-12 13:09:39', 'Entertainment', 1, 0.5299919247627258, '精品无码三级在线观看视频 极品熟妇大蝴蝶20P, 暴虐SM调教A片', '波多野结衣无码视频在线观看, 智能型鹽霧試驗, 精品无码三级在线观看视频, 林頻文化, 女人在厨房被添高潮全过程a片', 95.93928571428573, 'opinion')
('http://1156.herunchang.com/company/67137562', '2025-06-12 12:55:09', 'Lifestyle', 0, 0.6626068949699402, '公司数据更新中', '公司数据更新中, 兰坪县志华万盛种植农民专业合作社', 120.20500000000001, 'opinion')
('http://152.42.227.177/live-sgp/', '2025-06-12 13:55:03', 'Lifestyle', 1, 0.7075523138046265, 'Togel SGP, keputusan B. Meningkatkan, Nagasaon Live, Singapura B. Kecepatan, Nagasaon A. Pengenalan tentang, Togel A. Mempermudah, togel terpercaya, memberikan, Manfaat, menyajikan, Keunggulan Live Draw, Pentingnya Hasil Undian Terbaru A. Signifikansi, langsung ke hasil, dan akurat B. Peran Live Draw SGP dalam', 'sgp, pengambilan, pengenalan, langsung, signifikansi', 49.25185714285715, 'opinion')
('http://18teenporn.net/Sex-toys/', '2025-06-12 12:35:37', 'Entertainment', 1, 0.715

In [9]:
import pandas as pd
conn = sqlite3.connect('test_output.db')
df = pd.read_sql_query("select * from news_articles", conn)
conn.close()
df.head()

,url,url_timestamp,category,sentiment,score,named_entities,keywords,readability_score,opinion_type
0,http://0577007.com/gsjj/,2025-06-12 13:09:39,Entertainment,1,0.529992,"精品无码三级在线观看视频 极品熟妇大蝴蝶20P, 暴虐SM调教A片","波多野结衣无码视频在线观看, 智能型鹽霧試驗, 精品无码三级在线观看视频, 林頻文化, 女人...",95.939286,opinion
1,http://1156.herunchang.com/company/67137562,2025-06-12 12:55:09,Lifestyle,0,0.662607,公司数据更新中,"公司数据更新中, 兰坪县志华万盛种植农民专业合作社",120.205000,opinion
2,http://152.42.227.177/live-sgp/,2025-06-12 13:55:03,Lifestyle,1,0.707552,"Togel SGP, keputusan B. Meningkatkan, Nagasaon...","sgp, pengambilan, pengenalan, langsung, signif...",49.251857,opinion
3,http://18teenporn.net/Sex-toys/,2025-06-12 12:35:37,Entertainment,1,0.715802,Escorts,"escorts, toys, sex, 18teenporn, net",59.745000,opinion
4,http://205004.xobor.com/t1933f83-RIP-Lauren-Ba...,2025-06-12 12:51:22,Entertainment,0,0.996013,"RIP Lauren Bacall, RIP Lauren Bacall RIP Laure...","bacall, lauren, rip",48.525000,opinion
